use this notebook to see today's bet at the beginning of a day

In [26]:
import pandas as pd, numpy as np
import statsapi
from statsapi import player_stat_data
import requests
from datetime import datetime, timedelta
import numpy as np
import math
import meteostat
import pprint
import pickle

import pycaret
from pycaret import classification
import model.common
import importlib

In [19]:
from static_data.load_static_data import *

In [3]:
date_today = datetime.today().strftime("%Y-%m-%d")
date_yesterday = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
date_today, date_yesterday

('2023-05-29', '2023-05-28')

In [23]:
df_live_game_matchup = pd.read_pickle(f'collect_data/df_live_game_matchup_{date_today}.pkl')
df_odds_hits = pd.read_pickle(f"odds_data/df_odds_hits_{date_today}.pkl")

In [22]:
df_live_game_matchup[model.common.features + ['game_id']].set_index(['game_id', 'batting_name']).join(df_odds_hits.rename(columns={'player_name': 'batting_name'}).set_index(['game_id', 'batting_name']), lsuffix='', rsuffix='_odds')

pitching_gamesPlayed  pitching_runs  \
game_id batting_name                                             
717974  Adam Frazier                          4              3   
        Jorge Mateo                           4              3   
        Anthony Santander                     4              3   
        Ryan McKenna                          4              3   
        Ryan Mountcastle                      4              3   
...                                         ...            ...   
717978  Andrew McCutchen                      5             21   
        Tucupita Marcano                      5             21   
        Josh Palacios                         5             21   
        Jack Suwinski                         5             21   
        Ji Hwan Bae                           5             21   

                           pitching_strikeOuts  pitching_hits  pitching_id  \
game_id batting_name                                                         
717974  Adam Frazier                         6              9       663531   
        Jorge Mateo                          6              9       663531   
        Anthony Santander                    6              9       663531   
        Ryan McKenna                         6              9       663531   
        Ryan Mountcastle                     6              9       663531   
...                                        ...            ...          ...   
717978  Andrew McCutchen                    17             34       543101   
        Tucupita Marcano                    17             34       543101   
        Josh Palacios                       17             34       543101   
        Jack Suwinski                       17             34       543101   
        Ji Hwan Bae                         17             34       543101   

                           batting_gamesPlayed  batting_runs  \
game_id batting_name                                           
717974  Adam Frazier                       156            61   
        Jorge Mateo                        150            63   
        Anthony Santander                  152            78   
        Ryan McKenna                       105            23   
        Ryan Mountcastle                   145            62   
...                                        ...           ...   
717978  Andrew McCutchen                   134            66   
        Tucupita Marcano                    49            18   
        Josh Palacios                       29             8   
        Jack Suwinski                      106            45   
        Ji Hwan Bae                         10             5   

                           batting_strikeOuts  batting_hits  batting_rbi  ...  \
game_id batting_name                                                      ...   
717974  Adam Frazier                       73           129           42  ...   
        Jorge Mateo                       147           109           50  ...   
        Anthony Santander                 122           138           89  ...   
        Ryan McKenna                       55            37           11  ...   
        Ryan Mountcastle                  154           139           85  ...   
...                                       ...           ...          ...  ...   
717978  Andrew McCutchen                  124           122           69  ...   
        Tucupita Marcano                   44            33           13  ...   
        Josh Palacios                      15            10            2  ...   
        Jack Suwinski                     114            66           38  ...   
        Ji Hwan Bae                         6            11            6  ...   

                           game_date  game_year       pitching_name  \
game_id batting_name                                                  
717974  Adam Frazier      2023-05-29       2023         Logan Allen   
        Jorge Mateo       2023-05-29       2023      

# live prediction

In [27]:
regression_model = pycaret.classification.load_model(model.common.model_file_name)

Transformation Pipeline and Model Successfully Loaded


In [29]:
live_prediction = pycaret.classification.predict_model(data = df_live_game_matchup, estimator = regression_model)
live_prediction = pd.merge(live_prediction, df_player_team_positions[['player_id','player_team_name']], left_on='batting_id', right_on='player_id', how='left')
live_prediction["theo_odds"] = live_prediction["prediction_score"].apply(model.common.odds_calculator)

In [40]:
live_bet_columns = model.common.features + ['prediction_label', 'prediction_score', 'theo_odds']
live_prediction[live_bet_columns]

,pitching_gamesPlayed,pitching_runs,pitching_strikeOuts,pitching_hits,pitching_id,batting_gamesPlayed,batting_runs,batting_strikeOuts,batting_hits,batting_rbi,...,batting_id,temp,game_venue,game_date,game_year,batting_name,pitching_name,prediction_label,prediction_score,theo_odds
0,4,3,6,9,663531,156,61,73,129,42,...,624428,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Adam Frazier,Logan Allen,1,0.6346,-174
1,4,3,6,9,663531,156,61,73,129,42,...,624428,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Adam Frazier,Logan Allen,1,0.6346,-174
2,4,3,6,9,663531,156,61,73,129,42,...,624428,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Adam Frazier,Logan Allen,1,0.6346,-174
3,4,3,6,9,663531,156,61,73,129,42,...,624428,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Adam Frazier,Logan Allen,1,0.6346,-174
4,4,3,6,9,663531,150,63,147,109,50,...,622761,68.720001,Oriole Park at Camden Yards,2023-05-29,2023,Jorge Mateo,Logan Allen,1,0.5387,-117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,5,21,17,34,543101,49,18,44,33,13,...,672779,67.639999,Oracle Park,2023-05-29,2023,Tucupita Marcano,Anthony DeSclafani,1,0.5619,-128
163,5,21,17,34,543101,29,8,15,10,2,...,641943,67.639999,Oracle Park,2023-05-29,2023,Josh Palacios,Anthony DeSclafani,0,0.6221,-165
164,5,21,17,34,543101,29,8,15,10,2,...,641943,67.639999,Oracle Park,2023-05-29,2023,Josh Palacios,Anthony DeSclafani,0,0.6221,-165
165,5,21,17,34,543101,106,45,114,66,38,...,669261,67.639999,Oracle Park,2023-05-29,2023,Jack Suwinski,Anthony DeSclafani,1,0.5609,-128


In [41]:
df_live_prediction_odds = live_prediction[live_bet_columns + ['game_id']].set_index(['game_id', 'batting_name']).join(df_odds_hits.rename(columns={'player_name': 'batting_name'}).set_index(['game_id', 'batting_name']), lsuffix='', rsuffix='_odds')
df_live_prediction_odds

pitching_gamesPlayed  pitching_runs  \
game_id batting_name                                            
717968  Alex Bregman                        24             44   
        Jake Meyers                         24             44   
        Jeremy Pena                         24             44   
        Jose Abreu                          24             44   
        Jose Abreu                          24             44   
...                                        ...            ...   
717979  Travis Jankowski                    10              2   
        Zach McKinstry                      20             55   
        Zach McKinstry                      20             55   
        Zach McKinstry                      20             55   
        Zack Short                          20             55   

                          pitching_strikeOuts  pitching_hits  pitching_id  \
game_id batting_name                                                        
717968  Alex Bregman                      117             99       543243   
        Jake Meyers                       117             99       543243   
        Jeremy Pena                       117             99       543243   
        Jose Abreu                        117             99       543243   
        Jose Abreu                        117             99       543243   
...                                       ...            ...          ...   
717979  Travis Jankowski                   13              5       571510   
        Zach McKinstry                    103            115       543135   
        Zach McKinstry                    103            115       543135   
        Zach McKinstry                    103            115       543135   
        Zack Short                        103            115       543135   

                          batting_gamesPlayed  batting_runs  \
game_id batting_name                                          
717968  Alex Bregman                      155            93   
        Jake Meyers                        52            13   
        Jeremy Pena                       136            72   
        Jose Abreu                        157            85   
        Jose Abreu                        157            85   
...                                       ...           ...   
717979  Travis Jankowski                   44            11   
        Zach McKinstry                     57            21   
        Zach McKinstry                     57            21   
        Zach McKinstry                     57            21   
        Zack Short                          6             2   

                          batting_strikeOuts  batting_hits  batting_rbi  ...  \
game_id batting_name                                                     ...   
717968  Alex Bregman                      77           142           93  ...   
        Jake Meyers                       54            34           15  ...   
        Jeremy Pena                      135           132           63  ...   
        Jose Abreu                       110           183           75  ...   
        Jose Abreu                       110           183           75  ...   
...                                      ...           ...          ...  ...   
717979  Travis Jankowski                  10             9            2  ...   
        Zach McKinstry                    52            33           14  ...   
        Zach McKinstry                    52            33           14  ...   
        Zach McKinstry                    52            33           14  ...   
        Zack Short                         5             0            2  ...   

                          prediction_label  prediction_score  theo_odds  \
game_id batting_name                                                      
717968  Alex Bregman                     1            0.6747       -207   
        Jake Meyers                      0            0.5284       -112   
        Jeremy Pe

In [55]:
score_threshold = 0.70
df_live_prediction_odds_ = df_live_prediction_odds.reset_index()
confident_df_live_prediction_odds = df_live_prediction_odds_[(df_live_prediction_odds_["prediction_score"] >= score_threshold) & (df_live_prediction_odds_["prediction_label"] == 1)].sort_values(by = "prediction_score", ascending = False).drop_duplicates("batting_name")
confident_df_live_prediction_odds[['game_date', 'team_away', 'team_home', 'batting_name', "prediction_score", "theo_odds", 'over_odds', 'over_line']]

,game_date,team_away,team_home,batting_name,prediction_score,theo_odds,over_odds,over_line
5,2023-05-29,Minnesota Twins,Houston Astros,Jose Altuve,0.7475,-296,-260,0.5
124,2023-05-29,Texas Rangers,Detroit Tigers,Adolis Garcia,0.7387,-283,-230,0.5
4,2023-05-29,Minnesota Twins,Houston Astros,Jose Abreu,0.7370,-280,-260,0.5
93,2023-05-29,Colorado Rockies,Arizona Diamondbacks,Kris Bryant,0.7365,-280,-255,0.5
143,2023-05-29,Texas Rangers,Detroit Tigers,Marcus Semien,0.7352,-278,-270,0.5
128,2023-05-29,Texas Rangers,Detroit Tigers,Corey Seager,0.7281,-268,-260,0.5
148,2023-05-29,Texas Rangers,Detroit Tigers,Nathaniel Lowe,0.7182,-255,+195,1.5
132,2023-05-29,Texas Rangers,Detroit Tigers,Javier Baez,0.7135,-249,-275,0.5
62,2023-05-29,Kansas City Royals,St. Louis Cardinals,Paul Goldschmidt,0.7073,-242,-245,0.5
55,2023-05-29,Kansas City Royals,St. Louis Cardinals,Nolan Arenado,0.7002,-234,-275,0.5
